### Import

In [ ]:
import tkinter as tk

from tkinter import font as tkFont  
import matplotlib.pyplot as plt
import matplotlib.ticker as tkr 
from matplotlib.figure import Figure 
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg,  NavigationToolbar2Tk)  
import random
import pandas as pd
import pandastable 
from pandastable import Table
    
import numpy as np
import math as math

### Class DataManager
- Used to read the data from a datafile into a Pandas dataframe 
- Provides various functions that other classes may use to get hold of certain data

In [ ]:
class DataManager:

    def __init__(self): 
        self.df= None
        self.columns =None
        self.towns = None

    def readData(self,filename):
       
        self.df = pd.read_csv(filename )   
        self.columns = self.df.columns.tolist() 
        self.towns= self.df.loc[:,'town'].unique().tolist()  
        
        self.flatTypes= self.df.loc[:,'flat_type'].unique().tolist()

        # As for the one ROOM flat it is wrongly positioned at index 5, so we relocate it.
        newindex=0
        oldindex = 5
        self.flatTypes.insert(newindex, self.flatTypes.pop(oldindex)) 
 

    def getAveragePriceTown(self,town):
        dfTown = self.df[ self.df.loc[:,'town']==town] 
        noOfRows = len(dfTown)
        if noOfRows ==0:
            return 0
        total = dfTown.loc[:,'resale_price'].sum()
        avPrice = total/noOfRows
        avPrice = int(  round(avPrice,0))
        return avPrice

    def getMinMaxPriceTownType(self,town,type):
        dfTown = self.df[ (self.df.loc[:,'town']==town) & (self.df.loc[:,'flat_type']==type) ]
        min= dfTown.loc[:,'resale_price'].min()
        max= dfTown.loc[:,'resale_price'].max()
        return min,max

    def getPricesTownType(self,town,type):
        dfTownType = self.df[ (self.df.loc[:,'town']==town) & (self.df.loc[:,'flat_type']==type) ]  
        dfResalePrices = dfTownType.loc[:,'resale_price']  
        resalePrices = dfResalePrices.to_list()
        return resalePrices

    def getPricesTown(self,town):
        data = []
        for type in self.flatTypes: 
            resalePrices = self.getPricesTownType(town,type)
            data.append(resalePrices)
        return data
    
    def getFlatTypeAndTownByBudget(self,town,type,budgetMin,budgetMax):
        dfTown = self.df[ (self.df.loc[:,'town']==town) & (self.df.loc[:,'flat_type']==type) & (self.df.loc[:,'resale_price']>=budgetMin) & (self.df.loc[:,'resale_price']<=budgetMax) ]
        dfTown2 = dfTown.sort_values(by="resale_price")
        dfTown2=dfTown2.rename(columns={"month":"Date","block":"Block","storey_range":"Floor","floor_area_sqm":"Floor Area(sqm)","resale_price":"Price","street_name":"Street","remaining_lease":"Remaining Lease(yrs)"})
        return dfTown2
    
    
    
    


### Class ScreenManager
- Is used to magage loading/unloading the various screens

In [ ]:
class ScreenManager:

    def __init__(self, window,winWidth,winHeight, dataManager):  
        self.screenManager = self 
        
        # We need access to data
        self.dataManager = dataManager 
        
        # Our application various screens
        self.startScreen = StartScreen(window,winWidth,winHeight, self.screenManager)
        self.menuScreen = MenuScreen(window,winWidth,winHeight, self.screenManager)
        self.byTownScreen = ByTownScreen(window,winWidth,winHeight, self.screenManager)
        self.byTownCanvas = ByTownCanvas(window, self.byTownScreen.midRightFrame, # master
                                            winWidth,winHeight, self.screenManager) 
        self.byBudgetScreen = ByBudgetScreen(window,winWidth,winHeight, self.screenManager)
        self.byBudgetCanvas = ByBudgetCanvas(window, self.byBudgetScreen.midRightFrame, # master
                                            winWidth,winHeight, self.screenManager)
       
    def loadScreen(self, screen): 
        if screen == 'START_SCREEN':
            self.startScreen.frame.grid(row=0, column=0, padx = 0, pady = 0)
        elif screen == 'MENU_SCREEN':
            self.menuScreen.frame.grid(row=0, column=0, padx = 0, pady = 0) 
        elif screen == 'BY_TOWN_SCREEN':
            self.byTownScreen.load()
        elif screen == 'BY_BUDGET_SCREEN':
            self.byBudgetScreen.load()  
    
    def unLoadScreen(self,screen):
        if screen == 'START_SCREEN':
            self.startScreen.frame.grid_forget()
        elif  screen == 'MENU_SCREEN':
            self.menuScreen.frame.grid_forget()
        elif  screen == 'BY_TOWN_SCREEN':
            self.byTownScreen.forget() 
        elif  screen == 'BY_BUDGET_SCREEN':
            self.byBudgetScreen.forget()
          

### Class StartScreen
- Simple start screen with a Start button

In [ ]:
class StartScreen:

    def __init__(self,window,winWidth,winHeight,screenManager):

        # We need access to the ScreenManager
        self.screenManager = screenManager 
        
        # Set up a frame for our StartScreen
        self.frame =  tk.Frame(master=window,
                    width=winWidth,
                    height= winHeight , 
                    relief=tk.RAISED,
                    background = 'white',
                    borderwidth=1)      

        # Get ready all the widgets/items for this frame  
        self.helv36 = tkFont.Font(family='Helvetica',size=24, weight=tkFont.BOLD)
        screenTitleLabel = tk.Label(master=self.frame , text = 'Welcome to Resale Flat Prices Analyser',font = self.helv36, foreground='black', background ='white') 
        screenTitleLabel .place(x=0 ,y =0)  

        # Setup a Start Button and connect to a function (start)
        startButton = tk.Button(master=self.frame,text="Start",command=self.start, font =self.helv36) 
        startButton.place(x= 450 - 40, y=200)
    
    def start(self):   
        self.screenManager.unLoadScreen('START_SCREEN')
        self.screenManager.loadScreen('MENU_SCREEN') 



### Class MenuScreen
- Screen with Menu to help us navigate to various Screens

In [ ]:
class MenuScreen:

    def __init__(self,window,winWidth,winHeight,screenManager):
        
        # We need access to the ScreenManager and Window
        self.screenManager = screenManager
        self.window = window 

        # Setting a frame for our screen
        self.frame =  tk.Frame(master=window,
                    width=winWidth,
                    height= winHeight , 
                    relief=tk.RAISED,
                    background = 'white',
                    borderwidth=1)      

        # Get ready all the widgets/items for this frame  
        self.fontMenuTitle = tkFont.Font(family='Helvetica',size=24, weight=tkFont.BOLD)
        self.font15 = tkFont.Font(family='Helvetica',size=15, weight=tkFont.BOLD)
        
        menuTitleLabel = tk.Label(master=self.frame , text = 'Menu', 
                font = self.fontMenuTitle, foreground='black', background ='white') 
        menuTitleLabel.place(x=400 ,y =140) 
        
        # Some variables to control placement
        shiftLeft = 100 
        menuItemGap = 60
        
        # Make and place the buttons on the frame
        optionOneButton = tk.Button(master=self.frame,text="By Town",
                    command=self.optionOne, font =self.font15, height = 1, width = 15)  
        optionOneButton.place(x= 450 - shiftLeft, y=200)

        optionTwoButton = tk.Button(master=self.frame,text="By Budget",
                    command=self.optionTwo, font =self.font15, height = 1, width = 15) 
        optionTwoButton.place(x= 450 - shiftLeft, y=200 + menuItemGap)

        optionThreeButton = tk.Button(master=self.frame,text="Exit",
                    command=self.exit, font =self.font15, height = 1, width = 15) 
        optionThreeButton.place(x= 450 - shiftLeft, y=200 +2*menuItemGap)
    
    def optionOne(self): 
        self.screenManager.unLoadScreen('MENU_SCREEN') 
        self.screenManager.loadScreen('BY_TOWN_SCREEN')  
        
    def optionTwo(self): 
        self.screenManager.unLoadScreen('MENU_SCREEN') 
        self.screenManager.loadScreen('BY_BUDGET_SCREEN')  

    def exit(self):
        self.window.destroy()


### ByTownScreen
- This Screen will allow the uses to diplay boxplots of flat sale prices per town
- Screen is ogranized with various sub frame:
    - The screen contains three main sub frames (topFrame,midFrame and bottomFrame)
    - The frame in the middle is then splitted further into two sub frames (midLeftFrame and midRightFrame)
    - The ByTownCanvas (to display the boxplot) is then placed in the midRightFrame frame


In [ ]:
class ByTownScreen:

    def __init__(self,window,winWidth,winHeight,screenManager): 
    
        # We need access to Window, DataManager and ScreenManager
        self.window = window
        self.screenManager = screenManager   
        self.dataManager = self.screenManager.dataManager

        # Default, now adjustments needed
        self.winWidth = winWidth
        self.winHeight = winHeight  
        
        # Adjust Screen (if needed  to enlarge/shrink as tkinter may meddle with resolution)
        #self.winWidth = (int) (winWidth *  1.5)
        #self.winHeight = (int) (winHeight * 1.5)
        

        # Various stuff we need to build our frames
        self.fontTitle = tkFont.Font(family='Helvetica',size=24, weight=tkFont.BOLD)
        self.font12 = tkFont.Font(family='Helvetica',size=12, weight=tkFont.BOLD) 

        # Variable to control placement of widgest and frames
        self.TOP_FRAME_HEIGHT = 80
        self.BOTTOM_FRAME_HEIGHT = 50
        self.MID_FRAME_HEIGHT = self.winHeight - (self.TOP_FRAME_HEIGHT + self.BOTTOM_FRAME_HEIGHT)

        self.BACKGROUND_GREY = self.rgbTKinter(236,236,236)

        # Calculate width of graph frame as based on golden ratio
        goldenRatio = (1+ math.sqrt(5))/2 
        self.GRAPH_DISPLAY_FRAME_WIDTH= round(goldenRatio*self.MID_FRAME_HEIGHT,0) 
        self.GRAPH_CONTROL_FRAME_WIDTH= self.winWidth -  self.GRAPH_DISPLAY_FRAME_WIDTH

        # Our screen comprises three frames
        self.topFrame = self.createTopFrame()
        self.midFrame = self.createMidFrame()
        self.botFrame = self.createBotFrame() 
 
        # Our mid frame comprises two frames
        self.midLeftFrame = self.createMidLeftFrame()
        self.midLeftFrame.grid(row=0, column=0, padx = 0, pady = 0) 

        self.midRightFrame = self.createMidRightFrame()
        self.midRightFrame.grid(row=0, column=1, padx = 0, pady = 0)   
       
        
    def backToMenu(self): 
        self.screenManager.unLoadScreen('BY_TOWN_SCREEN') 
        self.screenManager.loadScreen('MENU_SCREEN')  

    def  handleTownMenu(self,choice):  
        self.selectedTown  = choice  
        self.screenManager.byTownCanvas.plot() 

    def createTopFrame(self):  

        topFrame =  tk.Frame(master=self.window,
                    width=self.winWidth,
                    height= self.TOP_FRAME_HEIGHT,
                    relief=tk.RAISED,
                    background = self.BACKGROUND_GREY,
                    borderwidth=2)      

        # Get ready all the widgets & items for this frame  
        optionTitleLabel = tk.Label(master=topFrame , text = 'Resale prices per town', 
        font = self.fontTitle, foreground='black', background =self.BACKGROUND_GREY) 
        optionTitleLabel.place(x=280 ,y = 10)   

        # Create a drop down menu for selecting the town 
        selectTownLabel = tk.Label(master=topFrame,font = self.font12,
            text="Select town:",background = self.BACKGROUND_GREY)  
        stringVar = tk.StringVar() 
    
        # The single star * unpacks the sequence/collectioninto positional arguments (starred expression)
        towns = self.screenManager.dataManager.towns
        stringVar.set(towns[0]) # Takes first Town in list
        self.selectedTown = towns[0]
        
        # Create the town selection drop down menu
        townMenu = tk.OptionMenu( topFrame, 
            stringVar,  *towns, 
            command=self.handleTownMenu)  
 
        # Place it
        selectTownLabel.place(x=665,y=25) 
        townMenu.place(x=765,y=25)   
        
        return topFrame

    def createMidFrame(self): # For Graph Display and Control 
        midFrame =  tk.Frame(master=self.window,
                    width=self.winWidth,
                    height= self.MID_FRAME_HEIGHT,  
                    relief=tk.RAISED,
                    background = 'white',
                    borderwidth=1)  
        return midFrame      

    def createMidLeftFrame(self): 
        midLeftFrame = tk.Frame(master=self.midFrame,
                    width=self.GRAPH_CONTROL_FRAME_WIDTH,
                    height= self.MID_FRAME_HEIGHT,  
                    relief=tk.RAISED,
                    background = self.BACKGROUND_GREY,
                    borderwidth=1) 

        # Setting up a legend for out boxplot by using a label
        legendText = "Flat types:\n----------------\n\n"
        for i in range( len(self.dataManager.flatTypes)):
            legendText+= str(i+1)+": " + self.dataManager.flatTypes[i] + "\n\n"  

        legendTextLabel = tk.Label(master=midLeftFrame,font = self.font12,
        text=legendText,background = self.BACKGROUND_GREY,  justify='left',anchor='w') 

        legendTextLabel.place(x=10,y=10)  
        return midLeftFrame


    def createMidRightFrame(self): 
        midRightFrame = tk.Frame(master=self.midFrame,
                    width=self.GRAPH_DISPLAY_FRAME_WIDTH,
                    height= self.MID_FRAME_HEIGHT,  
                    relief=tk.RAISED,
                    background = 'white',
                    borderwidth=1)
        return midRightFrame


    def createBotFrame(self): 
        botFrame =  tk.Frame(master=self.window,
                    width=self.winWidth,
                    height=self.BOTTOM_FRAME_HEIGHT,  
                    relief=tk.RAISED,
                    background = self.BACKGROUND_GREY,
                    borderwidth=2) 
        backToMenuButton = tk.Button(master=botFrame,text="Back to menu",
        command=self.backToMenu, font =self.font12) 
        backToMenuButton.place(x= 10, y=5)  
        return botFrame      

    def load(self):
        self.topFrame.grid(row=0, column=0, padx = 0, pady = 0) 
        self.midFrame.grid(row=1, column=0, padx = 0, pady = 0) 
        self.botFrame.grid(row=2, column=0, padx = 0, pady = 0)  
        self.screenManager.byTownCanvas.plot()

    def forget(self):
        self.topFrame.grid_forget()
        self.midFrame.grid_forget()
        self.botFrame.grid_forget()

    def rgbTKinter(self,r,g,b):   
        return f'#{r:02x}{g:02x}{b:02x}'
 
       

### Class ByTownCanvas
- Takes care of setting up a Canvas for data visualisation (in our to display the boxplot per town)
- The ByTownCanvas is placed in the midRightFrame frame of the ByTown screen

In [ ]:
class ByTownCanvas:

    def __init__(self,window,master,winWidth,winHeight,screenManager): 
    
        # We need access to Window, ScreenManager, DataManager 
        self.window = window 
        self.screenManager = screenManager   
        self.dataManager = self.screenManager.dataManager
        self.master = master

        self.winWidth =  winWidth
        self.winHeight = winHeight
        
        # Various stuff we need to build our frames
        self.fontTitle = tkFont.Font(family='Helvetica',size=24, weight=tkFont.BOLD)
        self.font12 = tkFont.Font(family='Helvetica',size=12, weight=tkFont.BOLD)

        # Get hold of figure and axis objects 
        fig, ax = plt.subplots(figsize=(5,3), dpi=120)  
        self.fig =fig
        self.ax = ax
        
        self.canvas = None

    def plotBoxplot(self,town):
            
        # Get the town that has been selected
        town = self.screenManager.byTownScreen.selectedTown

        # Get the data for the selected town  from our Data Manager
        data = self.dataManager.getPricesTown(town) 
        xlabels= self.dataManager.flatTypes 

        # Draw the box plot on the canvas
        self.ax.set_title(f'Box plot for: {town}', fontsize = 12, y = 1.05)   
        self.ax.set_yticks( np.arange(0, 1500000, 100000)   )
        self.ax.set_yticklabels(['100k','200k','300k','400k','500k','600k','700k','800k','900k'
                            ,'1000k','1100k','1200k','1300k','1400k','1500k'],rotation = 0)

        self.ax.boxplot(data) 

    
    def plot(self):   
      
        # Clear previous figure 
        
        plt.cla()
        self.plotBoxplot(self.screenManager.byTownScreen.selectedTown) 
        
        # Create canvas so as to display the Matplotlib figure  
        self.canvas = FigureCanvasTkAgg(self.fig, master = self.master) 
        self.canvas.draw() 
    
        # Placing the canvas on the Tkinter window 
        xPlace = 15 
        yPlace = 5 
        self.canvas.get_tk_widget().place(x=xPlace,y=yPlace) 
    



 ### Class ByBudget
- A screen for Budget

In [ ]:
class ByBudgetScreen:

    def __init__(self,window,winWidth,winHeight,screenManager): 
    
        # We need access to Window, DataManager and ScreenManager
        self.window = window
        self.screenManager = screenManager   
        self.dataManager = self.screenManager.dataManager

        # Default, now adjustments needed
        self.winWidth = winWidth
        self.winHeight = winHeight  
        
        # Adjust Screen (if needed  to enlarge/shrink as tkinter may meddle with resolution)
        #self.winWidth = (int) (winWidth *  1.5)
        #self.winHeight = (int) (winHeight * 1.5)
        

        # Various stuff we need to build our frames
        self.fontTitle = tkFont.Font(family='Helvetica',size=24, weight=tkFont.BOLD)
        self.font12 = tkFont.Font(family='Helvetica',size=12, weight=tkFont.BOLD) 
        self.font12_TNR = tkFont.Font(family='Lucida Console',size=12, weight=tkFont.BOLD) 

        # Variable to control placement of widgest and frames
        self.TOP_FRAME_HEIGHT = 80
        self.BOTTOM_FRAME_HEIGHT = 50
        self.MID_FRAME_HEIGHT = self.winHeight - (self.TOP_FRAME_HEIGHT + self.BOTTOM_FRAME_HEIGHT)

        self.BACKGROUND_GREY = self.rgbTKinter(236,236,236)

        # Calculate width of graph frame as based on golden ratio
        goldenRatio = (1+ math.sqrt(5))/2 
        self.GRAPH_DISPLAY_FRAME_WIDTH= round(goldenRatio*self.MID_FRAME_HEIGHT,0) 
        self.GRAPH_CONTROL_FRAME_WIDTH= self.winWidth -  self.GRAPH_DISPLAY_FRAME_WIDTH

        # Our screen comprises three frames
        self.topFrame = self.createTopFrame()
        self.midFrame = self.createMidFrame()
        self.botFrame = self.createBotFrame() 
 
        # Our mid frame comprises two frames
        self.midLeftFrame = self.createMidLeftFrame()
        self.midLeftFrame.grid(row=0, column=0, padx = 0, pady = 0) 

        self.midRightFrame = self.createMidRightFrame()
        self.midRightFrame.grid(row=0, column=1, padx = 0, pady = 0)   
       
        
    def backToMenu(self): 
        self.screenManager.unLoadScreen('BY_BUDGET_SCREEN') 
        self.screenManager.loadScreen('MENU_SCREEN')  

    def  handleTownMenu(self,choice):  
        
        for widgets in self.midRightFrame.winfo_children():
            widgets.destroy()
        
        self.selectedTown  = choice  
        data = self.dataManager.getFlatTypeAndTownByBudget(choice,self.selectedFlatType,
                                                           float(self.selectedMinBudget),float(self.selectedMaxBudget))
        
        root = self.midRightFrame
        
        if (len(data)>0):    
            #propertyList = str(data[['Mth','Block','Flr','Street','Floor Area(sqm)','Price']].to_string(index=False))
            data_new = pd.DataFrame(data)
            frame = tk.Frame(root)
            frame.pack(fill='both', expand = True)
            pt = Table(frame, dataframe = data_new )
            pt.show()
        else:
            msg = "No records found matching your search criteria."
            textLabel = tk.Label(master=self.midRightFrame,font = self.font12_TNR,
            text=msg,background = self.BACKGROUND_GREY,  justify='left',anchor='w') 
            textLabel.place(x=10,y=10)
            
        #textLabel = tk.Label(master=self.midRightFrame,font = self.font12_TNR,
        #text=propertyList,background = self.BACKGROUND_GREY,  justify='left',anchor='w') 

        #textLabel.place(x=10,y=10)  

    def handleFlatTypeMenu(self,choice):

        for widgets in self.midRightFrame.winfo_children():
            widgets.destroy()

        
        self.selectedFlatType = choice
 
        data = self.dataManager.getFlatTypeAndTownByBudget(self.selectedTown,choice,
                                                           float(self.selectedMinBudget),float(self.selectedMaxBudget))

        #if (len(data)>0):    
         #   propertyList = str(data[['Mth','Block','Flr','Street','Floor Area(sqm)','Price']].to_string(index=False))
        #else:
         #   propertyList = "No records found matching your search criteria."
                    
        #textLabel = tk.Label(master=self.midRightFrame,font = self.font12_TNR,
        #text=propertyList,background = self.BACKGROUND_GREY,  justify='left',anchor='w') 

        #textLabel.place(x=10,y=10)  
        root = self.midRightFrame
        
        if (len(data)>0):    
            #propertyList = str(data[['Mth','Block','Flr','Street','Floor Area(sqm)','Price']].to_string(index=False))
            data_new = pd.DataFrame(data)
            frame = tk.Frame(root)
            frame.pack(fill='both', expand = True)
            pt = Table(frame, dataframe = data_new )
            pt.show()
        else:
            msg = "No records found matching your search criteria."
            textLabel = tk.Label(master=self.midRightFrame,font = self.font12_TNR,
            text=msg,background = self.BACKGROUND_GREY,  justify='left',anchor='w') 
            textLabel.place(x=10,y=10)
        
    
    def handleMinBudgetLevelMenu(self,choice):
        for widgets in self.midRightFrame.winfo_children():
            widgets.destroy()

        data = self.dataManager.getFlatTypeAndTownByBudget(self.selectedTown,self.selectedFlatType,
                                                           float(choice),float(self.selectedMaxBudget))
        self.selectedMinBudget = choice
 
        

        #if (len(data)>0):    
         #   propertyList = str(data[['Mth','Block','Flr','Street','Floor Area(sqm)','Price']].to_string(index=False))
        #else:
         #   propertyList = "No records found matching your search criteria."
                    
        #textLabel = tk.Label(master=self.midRightFrame,font = self.font12_TNR,
        #text=propertyList,background = self.BACKGROUND_GREY,  justify='left',anchor='w') 

        
        
        root = self.midRightFrame
        
        if (len(data)>0):    
            #propertyList = str(data[['Mth','Block','Flr','Street','Floor Area(sqm)','Price']].to_string(index=False))
            data_new = pd.DataFrame(data)
            frame = tk.Frame(root)
            frame.pack(fill='both', expand = True)
            pt = Table(frame, dataframe = data_new )
            pt.show()
        else:
            msg = "No records found matching your search criteria."
            textLabel = tk.Label(master=self.midRightFrame,font = self.font12_TNR,
            text=msg,background = self.BACKGROUND_GREY,  justify='left',anchor='w') 
            textLabel.place(x=10,y=10)
            textLabel.place(x=10,y=10)
        
        
    def handleMaxBudgetLevelMenu(self,choice):
        for widgets in self.midRightFrame.winfo_children():
            widgets.destroy()

        
        self.selectedMaxBudget = choice

        root = self.midRightFrame
        data = self.dataManager.getFlatTypeAndTownByBudget(self.selectedTown,self.selectedFlatType,
                                                           float(choice),float(self.selectedMaxBudget))
        if (len(data)>0):    
            #propertyList = str(data[['Mth','Block','Flr','Street','Floor Area(sqm)','Price']].to_string(index=False))
            data_new = pd.DataFrame(data)
            frame = tk.Frame(root)
            frame.pack(fill='both', expand = True)
            pt = Table(frame, dataframe = data_new )
            pt.show()
        else:
            msg = "No records found matching your search criteria."
            textLabel = tk.Label(master=self.midRightFrame,font = self.font12_TNR,
            text=msg,background = self.BACKGROUND_GREY,  justify='left',anchor='w') 
            textLabel.place(x=10,y=10)
      
        
    def createTopFrame(self):  

        topFrame =  tk.Frame(master=self.window,
                    width=self.winWidth,
                    height= self.TOP_FRAME_HEIGHT,
                    relief=tk.RAISED,
                    background = self.BACKGROUND_GREY,
                    borderwidth=2)      

        # Get ready all the widgets & items for this frame  
        optionTitleLabel = tk.Label(master=topFrame , text = 'Resale prices by Budget', 
        font = self.fontTitle, foreground='black', background =self.BACKGROUND_GREY) 
        optionTitleLabel.place(x=280 ,y = 10)   

        # Create a drop down menu for selecting the town 
        selectTownLabel = tk.Label(master=topFrame,font = self.font12,
            text="Select Town:",background = self.BACKGROUND_GREY)  
        stringVar = tk.StringVar() 
    
        # The single star * unpacks the sequence/collectioninto positional arguments (starred expression)
        towns = self.screenManager.dataManager.towns
        stringVar.set(towns[0]) # Takes first Town in list
        self.selectedTown = towns[0]
        
        # Create the town selection drop down menu
        townMenu = tk.OptionMenu( topFrame, 
            stringVar,  *towns, 
            command=self.handleTownMenu)  
 
        # Place it
        selectTownLabel.place(x=665,y=25) 
        townMenu.place(x=765,y=25)   
        
        return topFrame

    def createMidFrame(self): # For Graph Display and Control 
        midFrame =  tk.Frame(master=self.window,
                    width=self.winWidth,
                    height= self.MID_FRAME_HEIGHT,  
                    relief=tk.RAISED,
                    background = 'white',
                    borderwidth=1)  
        return midFrame      

    def createMidLeftFrame(self): 
        midLeftFrame = tk.Frame(master=self.midFrame,
                    width=self.GRAPH_CONTROL_FRAME_WIDTH,
                    height= self.MID_FRAME_HEIGHT,  
                    relief=tk.RAISED,
                    background = self.BACKGROUND_GREY,
                    borderwidth=1) 

        selectFlatTypeLabel = tk.Label(master=midLeftFrame,font = self.font12,
        text="Flat Type:",background = self.BACKGROUND_GREY)  
        
        selectBudgetLabel = tk.Label(master=midLeftFrame,font = self.font12,
        text="Budget:",background = self.BACKGROUND_GREY)  
                
        selectMinBudgetLabel = tk.Label(master=midLeftFrame,font = self.font12,
        text="Min:",background = self.BACKGROUND_GREY)  

        selectMaxBudgetLabel = tk.Label(master=midLeftFrame,font = self.font12,
        text="Max:",background = self.BACKGROUND_GREY)  
        
        stringVar = tk.StringVar() 
        
        # for max budget
        stringVar2 = tk.StringVar() 
        
        # for min budget
        stringVar3 = tk.StringVar() 

        # The single star * unpacks the sequence/collectioninto positional arguments (starred expression)
        flatTypes = self.screenManager.dataManager.flatTypes
        
        budgetLevels = []
        for m in np.arange(50000, 1000000, 50000):
            budgetLevels.append(str(m))
            
        stringVar.set(flatTypes[0]) # Takes first Town in list
        
        stringVar2.set(budgetLevels[7]) # set initial max budget at $400k
        stringVar3.set(budgetLevels[1]) # set initial min budget at $100k
        
        self.selectedMinBudget = budgetLevels[1]
        self.selectedMaxBudget = budgetLevels[7]
        
        self.selectedFlatType = flatTypes[0]
        #self.selectedTown = towns[0]
        
        # Create the town selection drop down menu
        flatTypeMenu = tk.OptionMenu( midLeftFrame, 
            stringVar,  *flatTypes, 
            command=self.handleFlatTypeMenu)  

        maxBudgetLevelMenu = tk.OptionMenu( midLeftFrame, 
            stringVar2,  *budgetLevels, 
            command=self.handleMaxBudgetLevelMenu)  

        minBudgetLevelMenu = tk.OptionMenu( midLeftFrame, 
            stringVar3,  *budgetLevels, 
            command=self.handleMinBudgetLevelMenu)  

        
        
        # Place it
        selectFlatTypeLabel.place(x=10,y=10)         
        flatTypeMenu.place(x=110,y=5)                  

        selectBudgetLabel.place(x=10,y=80)         

        selectMinBudgetLabel.place(x=10,y=110)         
        minBudgetLevelMenu.place(x=50,y=110)                  

        selectMaxBudgetLabel.place(x=150,y=110)         
        maxBudgetLevelMenu.place(x=200,y=110)                  

                
        return midLeftFrame


    def createMidRightFrame(self): 
        midRightFrame = tk.Frame(master=self.midFrame,
                    width=self.GRAPH_DISPLAY_FRAME_WIDTH,
                    height= self.MID_FRAME_HEIGHT,  
                    relief=tk.RAISED,
                    background = 'white',
                    borderwidth=1)
        return midRightFrame


    def createBotFrame(self): 
        botFrame =  tk.Frame(master=self.window,
                    width=self.winWidth,
                    height=self.BOTTOM_FRAME_HEIGHT,  
                    relief=tk.RAISED,
                    background = self.BACKGROUND_GREY,
                    borderwidth=2) 
        backToMenuButton = tk.Button(master=botFrame,text="Back to menu",
        command=self.backToMenu, font =self.font12) 
        backToMenuButton.place(x= 10, y=5)  
        return botFrame      

    def load(self):
        self.topFrame.grid(row=0, column=0, padx = 0, pady = 0) 
        self.midFrame.grid(row=1, column=0, padx = 0, pady = 0) 
        self.botFrame.grid(row=2, column=0, padx = 0, pady = 0)  
        #self.screenManager.byBudgetCanvas.plot()

    def forget(self):
        self.topFrame.grid_forget()
        self.midFrame.grid_forget()
        self.botFrame.grid_forget()

    def rgbTKinter(self,r,g,b):   
        return f'#{r:02x}{g:02x}{b:02x}'
 




In [ ]:
class ByBudgetCanvas:

    def __init__(self,window,master,winWidth,winHeight,screenManager): 
    
        # We need access to Window, ScreenManager, DataManager 
        self.window = window 
        self.screenManager = screenManager   
        self.dataManager = self.screenManager.dataManager
        self.master = master

        self.winWidth =  winWidth
        self.winHeight = winHeight
        
        # Various stuff we need to build our frames
        self.fontTitle = tkFont.Font(family='Helvetica',size=24, weight=tkFont.BOLD)
        self.font12 = tkFont.Font(family='Helvetica',size=12, weight=tkFont.BOLD)

        # Get hold of figure and axis objects 
        fig, ax = plt.subplots(figsize=(5,3), dpi=120)  
        self.fig =fig
        self.ax = ax
        
        self.canvas = None
    
    def plotBoxplot(self,town):
            
        # Get the town that has been selected
        town = self.screenManager.byBudgetScreen.selectedTown

        # Get the data for the selected town  from our Data Manager
        data = self.dataManager.getPricesTown(town) 
        xlabels= self.dataManager.flatTypes 

        # Draw the box plot on the canvas
        self.ax.set_title(f'Box plot for: {town}', fontsize = 12, y = 1.05)   
        self.ax.set_yticks( np.arange(0, 1500000, 100000)   )
        self.ax.set_yticklabels(['100k','200k','300k','400k','500k','600k','700k','800k','900k'
                            ,'1000k','1100k','1200k','1300k','1400k','1500k'],rotation = 0)

        self.ax.boxplot(data) 

    
    def plot(self):   
      
        # Clear previous figure 
        plt.cla()
        self.plotBoxplot(self.screenManager.byBudgetScreen.selectedTown) 
        
        # Create canvas so as to display the Matplotlib figure  
        self.canvas = FigureCanvasTkAgg(self.fig, master = self.master) 
        self.canvas.draw() 
    
        # Placing the canvas on the Tkinter window 
        xPlace = 15 
        yPlace = 5 
        self.canvas.get_tk_widget().place(x=xPlace,y=yPlace) 
    

### Main Programme
- Take note run all cells above, before running the main programm

In [ ]:
#-------------------------------------------------------------
# Function for setting up a tkinter window 
#-------------------------------------------------------------
def setUpTKinterWindow(winWidth,winHeight): 
    # Setting up our tkinter window
    window = tk.Tk()
    window.resizable(width=False, height=False) # Non resizable 
    window.title("Resale Flat Prices")
    return window

#-------------------------------------------------------------
# Main program
#-------------------------------------------------------------
 
# Size of our window
winWidth = 900
winHeight = 500

# Set up our tkinter window
window = setUpTKinterWindow(winWidth,winHeight) 
 
# Get our DataManager up
dataManager = DataManager()

# Make sure you downloaded the data file and placed in a folder named data (together were your notebook is)
dataManager.readData('C:/Users/Anusha Kannapiran/Documents/Anusha Developments/Projects/data/resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv')

# Get our ScreenManager up
screenManager = ScreenManager(window,winWidth,winHeight,dataManager) 
 
# Load the start screen
screenManager.loadScreen('START_SCREEN') 

# Run the main loop  
window.mainloop()